## Task Graph Run scheduled every hour graph run to show:
* dag structure
* different run statuses
* graph config parameter
* task return value
* condition on stream
* condition on predecessor
* retry attempts
* SNS Task Notifications
* Event Logging
* Query Tagging

In [ ]:
use role TASK_GRAPH_ROLE;
use schema TASK_GRAPH_DATABASE.TASK_GRAPH_SCHEMA;

--this will show in query hitory and warehouse utilization filtering
ALTER SESSION SET query_tag = '{"origin":"sf_hol","name":"ftg_demo","version":{"major":1, "minor":0},"attributes":{"is_quickstart":1, "source":"notebook", "vignette":"tasks"}}';
ALTER SESSION SET TIMEZONE = 'America/Los_Angeles';
ALTER SESSION SET LOG_LEVEL = INFO;

-- send to a slack channel
call send_slack_message('The Task Graph Demo is starting at ' || CURRENT_TIME);

In [ ]:
--- function to randomize runtime with 1/10 as outlier (twice as long)
create or replace function RUNTIME_WITH_OUTLIERS(REGULAR_RUNTIME NUMBER(6,0))
returns NUMBER(6,0)
language SQL
comment = 'for input and output as milliseconds'
as
$$
    select
        case when uniform(1, 10, random()) = 10 
            then cast((REGULAR_RUNTIME * 2 + (uniform(-10, 10, random()))/100 * REGULAR_RUNTIME) as NUMBER(6,0))
            else cast((REGULAR_RUNTIME     + (uniform(-10, 10, random()))/100 * REGULAR_RUNTIME) as NUMBER(6,0))
        end
$$
;

In [ ]:
--- function to keep a running total 
CREATE OR REPLACE FUNCTION sum_table (INPUT_NUMBER number)
  returns TABLE (running_total number)
  language python
  runtime_version = '3.9'
  handler = 'gen_sum_table'
as
$$

# Define handler class
class gen_sum_table :

  ## Define __init__ method ro initilize the variable
  def __init__(self) :    
    self._running_sum = 0
  
  ## Define process method
  def process(self, input_number: float) :
    # Increment running sum with data from the input row
    new_total = self._running_sum + input_number
    self._running_sum = new_total

    yield(new_total,)
  
$$
;

In [ ]:
--- test randomized value around 5000 miliseconds
select RUNTIME_WITH_OUTLIERS(5000);


In [ ]:
-- successful procedure 1 eith event logging
create or replace procedure DEMO_PROCEDURE_1()        
returns VARCHAR(16777216)
language SQL
execute as OWNER
as 
$$
    select system$wait(3);
$$;

In [ ]:
-- failing procedure at 1/2 attempts since THE_OLD_TABLE does not exist
create or replace procedure DEMO_PROCEDURE_2()        
returns VARCHAR(16777216)
language SQL
execute as OWNER
as 
$$
declare
    RANDOM_VALUE number(2,0);
begin
    RANDOM_VALUE := (select uniform(1, 2, random()));
    if (:RANDOM_VALUE = 2) 
        then select count(*) from THE_OLD_TABLE;
    end if;
    select SYSTEM$WAIT(2);
end
$$;

In [ ]:
alter task if exists DEMO_TASK_1 suspend;

---- successful root task running every hour during US business hours 
create or replace task DEMO_TASK_1 
warehouse = 'TASK_GRAPH_WH' 
comment = ' root task running every hour during US Pacific business hours'
schedule = 'USING CRON 15 8-18 * * MON-FRI America/Los_Angeles'
SUSPEND_TASK_AFTER_NUM_FAILURES = 0
TASK_AUTO_RETRY_ATTEMPTS = 2
--- adding AWS SNS notification integration
ERROR_INTEGRATION = anowlan_sns_notify_int
SUCCESS_INTEGRATION = anowlan_sns_notify_int

--- adding default config parameter for runtime duration multiplier
config = $${"RUNTIME_MULTIPLIER": 5}$$                 
as
    declare
    --- get runtime duration factor from graph config as integer
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');       
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 1000);     --- specify the median runtime in milliseconds
    begin
       select SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');                                      --- task will wait for a random duration with 1/10 being 2x as long
       call SYSTEM$SET_RETURN_VALUE('✅ All systems go in DEMO_TASK_1'); --- demo return value to show in the UI
       
       SYSTEM$LOG('INFO', 'DEMO_TASK_1: Event Log entry: root task sucessful!'); --demo INFO event logging
    end
;

In [ ]:
--- Finalizer TASK to check all tables
create or replace task DEMO_FINALIZER
warehouse = 'TASK_GRAPH_WH'
finalize = DEMO_TASK_1
as
    declare
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER'); 
        --- get runtime duration factor from graph config as integer
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 1000);      
        --- specify the median runtime in milliseconds
    begin
       select SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');                               
       --- task will wait for a random duration with 1/10 being twice as long
       call SYSTEM$SET_RETURN_VALUE('✅ All checks completed via DEMO_FINALIZER');
       SYSTEM$LOG('INFO', 'DEMO_FINALIZER: completed!');
       --- demo return value to show in the UI
    end
;

In [ ]:
-- successful task with random duration
create or replace task DEMO_TASK_2 
warehouse = 'TASK_GRAPH_WH' 
comment = 'successful task loading data into prod_stock_inv'
after
    DEMO_TASK_1 
as
    declare
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 3000);            --- specify the median runtime in milliseconds
    begin
       select SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');       --- task will wait for a random duration with 1/10 being twice as long
       
       create or replace table prod_stock_inv as select * from table(gen_prod_inv(100)) order by 1; --- demo loading data into prod_stock_inv
       
       call SYSTEM$SET_RETURN_VALUE(:RANDOM_RUNTIME||'table prod_stock_inv loaded with data');
    end
;

In [ ]:
--- successful task with random duration calling a stored procedure 
create or replace task DEMO_TASK_3 
warehouse = 'TASK_GRAPH_WH' 
comment = 'successful task loading data into cust_info after random wait'
after
    DEMO_TASK_1
as
    declare
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 4000);            --- specify the median runtime in milliseconds
    begin
        call DEMO_PROCEDURE_1();
        
        select SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');       --- task will wait for a random duration with 1/10 being twice as long
        
        create or replace table cust_info as select * from table(gen_cust_info(1000)) order by 1; --- demo loading data into cust_info
        
        call SYSTEM$SET_RETURN_VALUE(:RANDOM_RUNTIME||'data loaded into cust_info table');
    end
;

In [ ]:
-- successful task with random duration
create or replace task DEMO_TASK_4 
warehouse = 'TASK_GRAPH_WH' 
comment = 'successful task loading data into table salesdata'
after
    DEMO_TASK_2 
as
    declare
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 1000);            --- specify the median runtime in milliseconds
    begin
        select SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');       --- task will wait for a random duration with 1/10 being twice as long
        
        call SYSTEM$SET_RETURN_VALUE('Delay: '||:RANDOM_RUNTIME||' milliseconds');

        create or replace table salesdata as select * from table(gen_cust_purchase(10000,10)); -- loading data into table salesdata
                    
        --lets use the events table; see cell 1
        SYSTEM$LOG_INFO('DEMO_TASK_4 loading data into table salesdata COMPLETE!');
    end
;

In [ ]:
create or replace task DEMO_TASK_5 
comment = 'serverless task creating the customer_sales_data_history dynamic table'
after
    DEMO_TASK_1, DEMO_TASK_4 
as
    declare
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 200);            --- specify the median runtime in milliseconds
    begin
        select SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');       --- task will wait for a random duration with 1/10 being twice as long
         --lets create a dynamic table for sales history
        CREATE OR REPLACE DYNAMIC TABLE customer_sales_data_history
            LAG='DOWNSTREAM'
            WAREHOUSE=XSMALL_WH
                AS
                select 
                    s.custid as customer_id,
                    c.cname as customer_name,
                    s.purchase:"prodid"::number(5) as product_id,
                    s.purchase:"purchase_amount"::number(10) as saleprice,
                    s.purchase:"quantity"::number(5) as quantity,
                    s.purchase:"purchase_date"::date as salesdate
                from
                    cust_info c inner join salesdata s on c.custid = s.custid;
                    
        call SYSTEM$SET_RETURN_VALUE('Delay: '||:RANDOM_RUNTIME||' milliseconds');
    end
;

In [ ]:
--- successful task calling a system function to send a random return value 1/2/3

create or replace task DEMO_TASK_6 
warehouse = 'TASK_GRAPH_WH' 
comment = 'successful task calling a system function to send a random return value 1, 2 or 3'
after
    DEMO_TASK_3 
as
    declare
        RANDOM_VALUE varchar;
    begin
        RANDOM_VALUE := (select UNIFORM(1, 3, RANDOM()));
        case when :RANDOM_VALUE = 1
        then
            call SYSTEM$SET_RETURN_VALUE('✅ Quality Check Passed');
        else
            call SYSTEM$SET_RETURN_VALUE('⚠️ Quality Check Failed from random gen function in Task 6');
        end;
    end;
;

In [ ]:
--- successful task calling system function 

create or replace task DEMO_TASK_7 
warehouse = 'TASK_GRAPH_WH' 
comment = 'successful task calling 1 system function'
after
    DEMO_TASK_6 
as
    declare
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 4000);            --- specify the median runtime in milliseconds
    begin
        RANDOM_RUNTIME := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 5000);            --- specify the median runtime in milliseconds
       
        call SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');       --- task will wait for a random duration with 1/20 being twice as long
       
        call SYSTEM$SET_RETURN_VALUE('https://github.com/sfc-gh-anowlan/SONY-FTG-HOL');
    end
;

In [ ]:
--- skipped task when stream condition is not met

create or replace task DEMO_TASK_8 
warehouse = 'TASK_GRAPH_WH' 
comment ='skipped task when stream condition is not met'
after
    DEMO_TASK_7 
when 
    SYSTEM$STREAM_HAS_DATA('PROD_STOCK_INV') 
as
   begin
        CREATE OR REPLACE DYNAMIC TABLE PROD_INV_ALERT
        LAG = '1 MINUTE'
        WAREHOUSE=XSMALL_WH
        AS
        SELECT 
            S.PRODUCT_ID, 
            S.PRODUCT_NAME,CREATIONTIME AS LATEST_SALES_DATE,
            STOCK AS BEGINING_STOCK,
            SUM(S.QUANTITY) OVER (PARTITION BY S.PRODUCT_ID ORDER BY CREATIONTIME) TOTALUNITSOLD, 
            (STOCK - TOTALUNITSOLD) AS UNITSLEFT,
            ROUND(((STOCK-TOTALUNITSOLD)/STOCK) *100,2) PERCENT_UNITLEFT,
            CURRENT_TIMESTAMP() AS ROWCREATIONTIME
        FROM SALESREPORT S JOIN PROD_STOCK_INV ON PRODUCT_ID = PID
        QUALIFY ROW_NUMBER() OVER (PARTITION BY PRODUCT_ID ORDER BY CREATIONTIME DESC) = 1;
        
        SYSTEM$LOG('INFO', 'DEMO_TASK_8: CTAS PROD_INV_ALERT dynamic table complete');
    end
;

In [ ]:
--- failing task with first procedure succeeding and second procedure failing 1/4 cases

create or replace task DEMO_TASK_9 
warehouse = 'TASK_GRAPH_WH' 
comment = 'failing task with first procedure succeeding and second procedure failing 1/4 cases'
after
    DEMO_TASK_4 
as
    begin
        call DEMO_PROCEDURE_1();
                
        select SYSTEM$WAIT(3);
        
        call DEMO_PROCEDURE_2();
        

        --create new 
        CREATE OR REPLACE DYNAMIC TABLE salesreport
            LAG = '1 MINUTE'
            WAREHOUSE=XSMALL_WH
            AS
            Select
                t1.customer_id,
                t1.customer_name, 
                t1.product_id,
                p.pname as product_name,
                t1.saleprice,
                t1.quantity,
                (t1.saleprice/t1.quantity) as unitsalesprice,
                t1.salesdate as CreationTime,
                customer_id || '-' || t1.product_id  || '-' || t1.salesdate AS CUSTOMER_SK,
                LEAD(CreationTime) OVER (PARTITION BY t1.customer_id ORDER BY CreationTime ASC) AS END_TIME
            from 
                customer_sales_data_history t1 inner join prod_stock_inv p 
                on t1.product_id = p.pid;
        SYSTEM$LOG('INFO', 'DEMO_TASK_9: CTAS salesreport dynamic table complete');
    end
;

In [ ]:
--- task does not run after failing task 9

create or replace task DEMO_TASK_10 
warehouse = 'TASK_GRAPH_WH' 
comment = 'task does not run after failing task 9'
after
    DEMO_TASK_9 
as
    declare
    --- specify the median runtime in milliseconds
        RUNTIME_MULTIPLIER integer := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');
        RANDOM_RUNTIME varchar := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 4000);            
    begin
        RANDOM_RUNTIME := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 2000);            --- specify the median runtime in milliseconds
        select SYSTEM$WAIT(:RANDOM_RUNTIME,'MILLISECONDS');       --- task will wait for a random duration with 1/10 being twice as long
        
        return 'Delay: '||:RANDOM_RUNTIME||' milliseconds';
    end
;

In [ ]:
--- task skipped 1/3 times, if TASK_6 returns '3' 

create or replace task DEMO_TASK_11 
warehouse = 'TASK_GRAPH_WH' 
comment = 'task skipped 1/3 times, if TASK_6 returns passed'

after
  DEMO_TASK_6 as declare 
    PREDECESSOR_VALUE varchar;
    RUNTIME_MULTIPLIER integer;
    RANDOM_RUNTIME varchar;
begin PREDECESSOR_VALUE := SYSTEM$GET_PREDECESSOR_RETURN_VALUE('DEMO_TASK_6');
case
    when :PREDECESSOR_VALUE = '✅ Quality Check Passed' 
        then 
            RUNTIME_MULTIPLIER := SYSTEM$GET_TASK_GRAPH_CONFIG('RUNTIME_MULTIPLIER');
            RANDOM_RUNTIME := RUNTIME_WITH_OUTLIERS(:RUNTIME_MULTIPLIER * 3000);
            select
                SYSTEM$WAIT(:RANDOM_RUNTIME, 'MILLISECONDS');
                call SYSTEM$SET_RETURN_VALUE('Delay: ' || :RANDOM_RUNTIME || ' milliseconds');
        else 
            SYSTEM$LOG('ERROR', 'DEMO_TASK_11: ' || :PREDECESSOR_VALUE);
            call SYSTEM$SET_RETURN_VALUE('Task skipped due to failed quality check');
  end case;
  end;

In [ ]:
--- task self-cancelling 1/10 times after long run
create or replace task DEMO_TASK_12 
warehouse = 'TASK_GRAPH_WH'
comment = 'task self-cancelling 1/10 times after long run'
after
    DEMO_TASK_3 
as
    declare
        RANDOM_VALUE number(2,0);
    begin
        RANDOM_VALUE := (select UNIFORM(1, 10, RANDOM()));
        if (:RANDOM_VALUE = 10) then
            select SYSTEM$WAIT(12);
            SYSTEM$LOG('ERROR', 'DEMO_TASK_12: Canceling Execution');
            select SYSTEM$USER_TASK_CANCEL_ONGOING_EXECUTIONS('DEMO_TASK_12');
        end if;
        
        select SYSTEM$WAIT(2);
    end
;

In [ ]:
--- successful task with 2 predecessors
create or replace task DEMO_TASK_13 
warehouse = 'TASK_GRAPH_WH'
comment = 'successful task CTAS cumulative_purchase dynamic table'
after
    DEMO_TASK_12,
    DEMO_TASK_2
as

    CREATE OR REPLACE DYNAMIC TABLE cumulative_purchase
    LAG = '1 MINUTE'
    WAREHOUSE=XSMALL_WH
    AS
        select 
            month(creationtime) monthNum,
            year(creationtime) yearNum,
            customer_id, 
            saleprice,
            running_total 
        from 
            salesreport,
            table(sum_table(saleprice) over (partition by creationtime,customer_id order by creationtime, customer_id));
;

In [ ]:
--- always suspended task
create or replace task DEMO_TASK_14 
warehouse = 'TASK_GRAPH_WH'
comment = 'always suspended task'
after
    DEMO_TASK_9 
as
    select SYSTEM$WAIT(3)
;

In [ ]:
--- always suspended task
create or replace task DEMO_TASK_15 
warehouse = 'TASK_GRAPH_WH'
comment = 'never runs because predecessor is suspended'
after
    DEMO_TASK_14 
as
    select 1
;

In [ ]:
--- resume all, suspend 1 to suspend 14
alter task DEMO_TASK_1 suspend;
alter task DEMO_TASK_14 suspend;
alter task if exists DEMO_TASK_1 SET LOG_LEVEL = 'INFO';
select SYSTEM$TASK_DEPENDENTS_ENABLE('DEMO_TASK_1');

---  resume 1 and execute
alter task DEMO_TASK_1 resume;
execute task DEMO_TASK_1;

In [ ]:
-- select * from TASK_GRAPH_DATABASE.TASK_GRAPH_SCHEMA.events_in_task_graph WHERE resource_attributes:"snow.executable.type" = 'FAILED' AND    resource_attributes:"snow.schema.name" = 'MY_SCHEMA' AND    value:state = 'FAILED' and RECORD_TYPE='LOG' and cast(timestamp as date) > CURRENT_DATE;

---SELECT OBSERVED_TIMESTAMP, RECORD ,RECORD_ATTRIBUTES, VALUE, SCOPE 
--  FROM TASK_GRAPH_DATABASE.TASK_GRAPH_SCHEMA.events_in_task_graph
--  WHERE
--    RECORD_TYPE= 'LOG' AND 
--    resource_attributes:"snow.schema.name" = 'TASK_GRAPH_SCHEMA' 
--  ORDER BY observed_timestamp DESC LIMIT 100;
  
--select CURRENT_DATE;
--select * from Snowflake.Telemetry.events ;

select * from salesreport limit 10;
select count(*) from salesreport;

-- Add new records
insert into salesdata select * from table(gen_cust_purchase(10000,2));

-- Check raw base table
select count(*) from salesdata;

-- Check Dynamic Tables after a minute
select count(*) from customer_sales_data_history;
select count(*) from salesreport;

select * from  cumulative_purchase limit 10;

-- check products with low inventory and alert
select * from prod_inv_alert where percent_unitleft < 10;

Now make a change to the table/stream to see Task 8 execute

In [ ]:
CREATE OR REPLACE ALERT alert_low_inv
  WAREHOUSE = XSMALL_WH
  SCHEDULE = '30 MINUTE'
  IF (EXISTS (
      SELECT *
      FROM prod_inv_alert
      WHERE percent_unitleft < 10 and ROWCREATIONTIME > SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME()
  ))
  THEN CALL send_slack_message('Alert: Low Inventory of products. Inventory running low for certain products. Please check the inventory report in Snowflake table prod_inv_alert.');

-- Alerts are pause by default, so let's resume it first
ALTER ALERT alert_low_inv RESUME;

-- Add new records
insert into salesdata select * from table(gen_cust_purchase(10000,2));

In [ ]:
alter task DEMO_TASK_1 suspend;
alter task DEMO_TASK_1, DEMO_TASK_2,DEMO_TASK_3,DEMO_TASK_4,DEMO_TASK_5,DEMO_TASK_8,DEMO_TASK_13 SET LOG_LEVEL = 'INFO';

---  resume 1 and execute ALL
select SYSTEM$TASK_DEPENDENTS_ENABLE('DEMO_TASK_1');

alter task DEMO_TASK_1 resume;
execute task DEMO_TASK_1;